## 1. `Importing Libraries📦`

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sns

import optuna
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import GridSearchCV, train_test_split, RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import GradientBoostingRegressor, HistGradientBoostingRegressor
from sklearn.metrics import roc_curve, mean_absolute_error, mean_squared_error
from xgboost import XGBRegressor, XGBRFRegressor
from catboost import CatBoostRegressor

import torch
import torch.nn as nn
import torch.optim as optim
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP

In [2]:
# print("CUDA Available:", torch.cuda.is_available())
# print("Number of GPUs:", torch.cuda.device_count())
# if torch.cuda.is_available():
#     for i in range(torch.cuda.device_count()):
#         print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
# print("Current Device:", torch.cuda.current_device())

## 2. `Importing Dataset📂`

In [3]:
train_data = pd.read_csv('/kaggle/input/playground-prediction-competition-dataset-2/train_data.csv')
test_data = pd.read_csv('/kaggle/input/playground-prediction-competition-dataset-2/test_data.csv')

In [4]:
train_data.head()

,id,Compartments,Weight Capacity (kg),Price,Brand_Adidas,Brand_Jansport,Brand_Nike,Brand_Puma,Brand_Under Armour,Material_Canvas,...,Waterproof_Yes,Style_Backpack,Style_Messenger,Style_Tote,Color_Black,Color_Blue,Color_Gray,Color_Green,Color_Pink,Color_Red
0,0,7.0,11.611723,112.15875,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1,10.0,27.078537,68.88056,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2,2.0,16.643760,39.17320,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,3,8.0,12.937220,80.60793,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,4,1.0,17.749338,86.02312,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [5]:
test_data.head()

,id,Compartments,Weight Capacity (kg),Brand_Adidas,Brand_Jansport,Brand_Nike,Brand_Puma,Brand_Under Armour,Material_Canvas,Material_Leather,...,Waterproof_Yes,Style_Backpack,Style_Messenger,Style_Tote,Color_Black,Color_Blue,Color_Gray,Color_Green,Color_Pink,Color_Red
0,300000,2.0,20.671147,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,300001,7.0,13.564105,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,300002,9.0,11.809799,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,300003,1.0,18.477036,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,300004,2.0,9.907953,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


## 3. `DataSet Splittting`

In [6]:
X, Y = train_data.drop(['Price', 'id'], axis = 1), train_data.Price
test_X = test_data.iloc[:, 1:]

In [7]:
X.head()

,Compartments,Weight Capacity (kg),Brand_Adidas,Brand_Jansport,Brand_Nike,Brand_Puma,Brand_Under Armour,Material_Canvas,Material_Leather,Material_Nylon,...,Waterproof_Yes,Style_Backpack,Style_Messenger,Style_Tote,Color_Black,Color_Blue,Color_Gray,Color_Green,Color_Pink,Color_Red
0,7.0,11.611723,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,10.0,27.078537,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2.0,16.643760,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,8.0,12.937220,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1.0,17.749338,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [8]:
Y.head()

0    112.15875
1     68.88056
2     39.17320
3     80.60793
4     86.02312
Name: Price, dtype: float64

In [9]:
test_X.head()

,Compartments,Weight Capacity (kg),Brand_Adidas,Brand_Jansport,Brand_Nike,Brand_Puma,Brand_Under Armour,Material_Canvas,Material_Leather,Material_Nylon,...,Waterproof_Yes,Style_Backpack,Style_Messenger,Style_Tote,Color_Black,Color_Blue,Color_Gray,Color_Green,Color_Pink,Color_Red
0,2.0,20.671147,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,7.0,13.564105,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,9.0,11.809799,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1.0,18.477036,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,2.0,9.907953,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25)

#### Normalizing Numerical Cols

In [11]:
X.nunique()

Compartments                   10
Weight Capacity (kg)      1920345
Brand_Adidas                    2
Brand_Jansport                  2
Brand_Nike                      2
Brand_Puma                      2
Brand_Under Armour              2
Material_Canvas                 2
Material_Leather                2
Material_Nylon                  2
Material_Polyester              2
Size_Large                      2
Size_Medium                     2
Size_Small                      2
Laptop Compartment_No           2
Laptop Compartment_Yes          2
Waterproof_No                   2
Waterproof_Yes                  2
Style_Backpack                  2
Style_Messenger                 2
Style_Tote                      2
Color_Black                     2
Color_Blue                      2
Color_Gray                      2
Color_Green                     2
Color_Pink                      2
Color_Red                       2
dtype: int64

In [12]:
ss = StandardScaler()
X_train['Weight Capacity (kg)'] = ss.fit_transform(X_train[['Weight Capacity (kg)']])
X_test['Weight Capacity (kg)'] = ss.transform(X_test[['Weight Capacity (kg)']])

In [13]:
X_train.head()

,Compartments,Weight Capacity (kg),Brand_Adidas,Brand_Jansport,Brand_Nike,Brand_Puma,Brand_Under Armour,Material_Canvas,Material_Leather,Material_Nylon,...,Waterproof_Yes,Style_Backpack,Style_Messenger,Style_Tote,Color_Black,Color_Blue,Color_Gray,Color_Green,Color_Pink,Color_Red
3971831,6.0,-0.079710,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3231464,8.0,-1.312085,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
254739,1.0,-0.391180,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2293358,2.0,-0.950520,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
310851,3.0,-1.278069,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [14]:
X_test.head()

,Compartments,Weight Capacity (kg),Brand_Adidas,Brand_Jansport,Brand_Nike,Brand_Puma,Brand_Under Armour,Material_Canvas,Material_Leather,Material_Nylon,...,Waterproof_Yes,Style_Backpack,Style_Messenger,Style_Tote,Color_Black,Color_Blue,Color_Gray,Color_Green,Color_Pink,Color_Red
209210,8.0,-0.620798,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
881629,1.0,-0.648109,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2138574,3.0,-0.320703,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3968856,8.0,-1.474975,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1932750,4.0,0.008919,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


## 4. `Model Training🤖`

In [15]:
# dtr = DecisionTreeRegressor()
# dtr.fit(X_train, Y_train)

# train_pred = dtr.predict(X_train)
# test_pred = dtr.predict(X_test)

# train_rmse = (mean_squared_error(Y_train, train_pred) ** 0.5)
# test_rmse = (mean_squared_error(Y_test, test_pred) ** 0.5)

# print(f"Train RMSE: {train_rmse:.4f}")
# print(f"Test RMSE: {test_rmse:.4f}")

# fig, axes = plt.subplots(1, 2, figsize=(12, 5)) 

# axes[0].scatter(Y_train, train_pred, marker='.', color='skyblue', alpha=0.6)
# axes[0].plot([min(Y_train), max(Y_train)], [min(Y_train), max(Y_train)], color="red", linestyle="--")  # Ideal line
# axes[0].set_xlabel("True Values (Train)")
# axes[0].set_ylabel("Predicted Values (Train)")
# axes[0].set_title(f"Train Evaluation (RMSE: {train_rmse:.4f})")

# # Test set plot
# axes[1].scatter(Y_test, test_pred, marker='.', color='lightcoral', alpha=0.6)
# axes[1].plot([min(Y_test), max(Y_test)], [min(Y_test), max(Y_test)], color="black", linestyle="--")  # Ideal line
# axes[1].set_xlabel("True Values (Test)")
# axes[1].set_ylabel("Predicted Values (Test)")
# axes[1].set_title(f"Test Evaluation (RMSE: {test_rmse:.4f})")

# plt.tight_layout()
# plt.show()

In [16]:
# gbr = GradientBoostingRegressor()
# gbr.fit(X_train, Y_train)

# train_pred = gbr.predict(X_train)
# test_pred = gbr.predict(X_test)

# train_rmse = mean_squared_error(Y_train, train_pred, squared=False)  # RMSE
# test_rmse = mean_squared_error(Y_test, test_pred, squared=False)  # RMSE

# print(f"Train RMSE: {train_rmse:.4f}")
# print(f"Test RMSE: {test_rmse:.4f}")

# # Visualization
# fig, axes = plt.subplots(1, 2, figsize=(12, 5))  # Two subplots (side by side)

# # Train set plot
# axes[0].scatter(Y_train, train_pred, marker='.', color='skyblue', alpha=0.6)
# axes[0].plot([min(Y_train), max(Y_train)], [min(Y_train), max(Y_train)], color="red", linestyle="--")  # Ideal line
# axes[0].set_xlabel("True Values (Train)")
# axes[0].set_ylabel("Predicted Values (Train)")
# axes[0].set_title(f"Train Evaluation (RMSE: {train_rmse:.4f})")

# # Test set plot
# axes[1].scatter(Y_test, test_pred, marker='.', color='lightcoral', alpha=0.6)
# axes[1].plot([min(Y_test), max(Y_test)], [min(Y_test), max(Y_test)], color="red", linestyle="--")  # Ideal line
# axes[1].set_xlabel("True Values (Test)")
# axes[1].set_ylabel("Predicted Values (Test)")
# axes[1].set_title(f"Test Evaluation (RMSE: {test_rmse:.4f})")

# plt.tight_layout()
# plt.show()

In [17]:
# xgb = XGBRegressor()
# xgb.fit(X_train, Y_train)

# # Predictions
# train_pred = xgb.predict(X_train)
# test_pred = xgb.predict(X_test)

# # RMSE Calculation
# train_rmse = mean_squared_error(Y_train, train_pred, squared=False)  # RMSE
# test_rmse = mean_squared_error(Y_test, test_pred, squared=False)  # RMSE

# print(f"Train RMSE: {train_rmse:.4f}")
# print(f"Test RMSE: {test_rmse:.4f}")

# # Visualization
# fig, axes = plt.subplots(1, 2, figsize=(12, 5))  # Two subplots (side by side)

# # Train set plot
# axes[0].scatter(Y_train, train_pred, marker='.', color='skyblue', alpha=0.6)
# axes[0].plot([min(Y_train), max(Y_train)], [min(Y_train), max(Y_train)], color="red", linestyle="--")  # Ideal line
# axes[0].set_xlabel("True Values (Train)")
# axes[0].set_ylabel("Predicted Values (Train)")
# axes[0].set_title(f"Train Evaluation (RMSE: {train_rmse:.4f})")

# # Test set plot
# axes[1].scatter(Y_test, test_pred, marker='.', color='lightcoral', alpha=0.6)
# axes[1].plot([min(Y_test), max(Y_test)], [min(Y_test), max(Y_test)], color="red", linestyle="--")  # Ideal line
# axes[1].set_xlabel("True Values (Test)")
# axes[1].set_ylabel("Predicted Values (Test)")
# axes[1].set_title(f"Test Evaluation (RMSE: {test_rmse:.4f})")

# plt.tight_layout()
# plt.show()

In [18]:
# cbr = CatBoostRegressor(verbose=0)  # Suppress training logs
# cbr.fit(X_train, Y_train)

# # Predictions
# train_pred = cbr.predict(X_train)
# test_pred = cbr.predict(X_test)

# # RMSE Calculation
# train_rmse = mean_squared_error(Y_train, train_pred, squared=False)  # RMSE
# test_rmse = mean_squared_error(Y_test, test_pred, squared=False)  # RMSE

# print(f"Train RMSE: {train_rmse:.4f}")
# print(f"Test RMSE: {test_rmse:.4f}")

# # Visualization
# fig, axes = plt.subplots(1, 2, figsize=(12, 5))  # Two subplots (side by side)

# # Train set plot
# axes[0].scatter(Y_train, train_pred, marker='.', color='skyblue', alpha=0.6)
# axes[0].plot([min(Y_train), max(Y_train)], [min(Y_train), max(Y_train)], color="red", linestyle="--")  # Ideal line
# axes[0].set_xlabel("True Values (Train)")
# axes[0].set_ylabel("Predicted Values (Train)")
# axes[0].set_title(f"Train Evaluation (RMSE: {train_rmse:.4f})")

# # Test set plot
# axes[1].scatter(Y_test, test_pred, marker='.', color='lightcoral', alpha=0.6)
# axes[1].plot([min(Y_test), max(Y_test)], [min(Y_test), max(Y_test)], color="red", linestyle="--")  # Ideal line
# axes[1].set_xlabel("True Values (Test)")
# axes[1].set_ylabel("Predicted Values (Test)")
# axes[1].set_title(f"Test Evaluation (RMSE: {test_rmse:.4f})")

# plt.tight_layout()
# plt.show()

In [ ]:
param_dist = {
    'n_estimators': [100, 500, 1000, 1500],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 6, 9, 12],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bynode': [0.6, 0.8, 1.0],
    'random_state': [42]
}

xgbrf = XGBRFRegressor()

random_search = RandomizedSearchCV(
    estimator=xgbrf,
    param_distributions=param_dist,
    scoring='neg_mean_squared_error',
    n_iter=20,  # Number of iterations
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1,
    random_state=42
)

random_search.fit(X_train, Y_train)

best_params = random_search.best_params_
print("Best Parameters:", best_params)

xgbrf_best = XGBRFRegressor(**best_params)
xgbrf_best.fit(X_train, Y_train)

train_pred = xgbrf_best.predict(X_train)
test_pred = xgbrf_best.predict(X_test)

train_rmse = mean_squared_error(Y_train, train_pred, squared=False)
test_rmse = mean_squared_error(Y_test, test_pred, squared=False)

print(f"Train RMSE: {train_rmse:.4f}")
print(f"Test RMSE: {test_rmse:.4f}")

Fitting 5 folds for each of 20 candidates, totalling 100 fits


In [ ]:
param_dist = {
    'max_iter': [1000, 5000, 10000],
    'learning_rate': [0.001, 0.01, 0.1],
    'max_leaf_nodes': [None, 10, 20, 50],
    'n_iter_no_change': [100, 500, 1000]
}

hgbr = HistGradientBoostingRegressor()


random_search = RandomizedSearchCV(
    estimator=hgbr,
    param_distributions=param_dist,
    scoring='neg_mean_squared_error',
    n_iter=20,  # Number of iterations
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1,
    random_state=42
)

random_search.fit(X_train, Y_train)

best_params = random_search.best_params_
print("Best Parameters:", best_params)

hgbr_best = HistGradientBoostingRegressor(**best_params)
hgbr_best.fit(X_train, Y_train)

train_pred = hgbr_best.predict(X_train)
test_pred = hgbr_best.predict(X_test)

train_rmse = mean_squared_error(Y_train, train_pred, squared=False)
test_rmse = mean_squared_error(Y_test, test_pred, squared=False)

print(f"Train RMSE: {train_rmse:.4f}")
print(f"Test RMSE: {test_rmse:.4f}")

fig, axes = plt.subplots(1, 2, figsize=(12, 5))  # Two subplots (side by side)

axes[0].scatter(Y_train, train_pred, marker='.', color='skyblue', alpha=0.6)
axes[0].plot([min(Y_train), max(Y_train)], [min(Y_train), max(Y_train)], color="red", linestyle="--")  # Ideal line
axes[0].set_xlabel("True Values (Train)")
axes[0].set_ylabel("Predicted Values (Train)")
axes[0].set_title(f"Train Evaluation (RMSE: {train_rmse:.4f})")

axes[1].scatter(Y_test, test_pred, marker='.', color='lightcoral', alpha=0.6)
axes[1].plot([min(Y_test), max(Y_test)], [min(Y_test), max(Y_test)], color="red", linestyle="--")  # Ideal line
axes[1].set_xlabel("True Values (Test)")
axes[1].set_ylabel("Predicted Values (Test)")
axes[1].set_title(f"Test Evaluation (RMSE: {test_rmse:.4f})")

plt.tight_layout()
plt.show()

- the best model is HistGradientBoostingRegressor

## -> `Training NN in Pytorch`

In [21]:
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32).to('cuda')
y_train_tensor = torch.tensor(Y_train.values, dtype=torch.float32).view(-1, 1).to('cuda')
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32).to('cuda')
y_test_tensor = torch.tensor(Y_test.values, dtype=torch.float32).view(-1, 1).to('cuda')

In [38]:
class RegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.bn1 = nn.BatchNorm1d(128)
        
        self.fc2 = nn.Linear(128, 256)
        self.bn2 = nn.BatchNorm1d(256)

        self.fc3 = nn.Linear(256, 128)
        self.bn3 = nn.BatchNorm1d(128)

        self.fc4 = nn.Linear(128, 128)
        self.bn4 = nn.BatchNorm1d(128)

        self.fc5 = nn.Linear(128, 64)
        self.bn5 = nn.BatchNorm1d(64)

        self.output = nn.Linear(64, 1)

        self.dropout = nn.Dropout(0.2)
        self.activation = nn.LeakyReLU(0.1)

    def forward(self, x):
        x = self.activation(self.bn1(self.fc1(x)))
        x = self.dropout(x)

        x = self.activation(self.bn2(self.fc2(x)))
        x = self.dropout(x)

        x = self.activation(self.bn3(self.fc3(x)))
        x = self.dropout(x)

        x = self.activation(self.bn4(self.fc4(x)))
        x = self.dropout(x)

        x = self.activation(self.bn5(self.fc5(x)))
        return self.output(x)

input_dim = X_train.shape[1]
model = RegressionModel(input_dim).to('cuda')

if torch.cuda.device_count() > 1:
    print("Using", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)

def rmse_loss(pred, target):
    return torch.sqrt(nn.MSELoss()(pred, target))

optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)  # Reduce LR every 10 epochs

print(model)  

Using 2 GPUs!
DataParallel(
  (module): RegressionModel(
    (fc1): Linear(in_features=27, out_features=128, bias=True)
    (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (fc2): Linear(in_features=128, out_features=256, bias=True)
    (bn2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (fc3): Linear(in_features=256, out_features=128, bias=True)
    (bn3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (fc4): Linear(in_features=128, out_features=128, bias=True)
    (bn4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (fc5): Linear(in_features=128, out_features=64, bias=True)
    (bn5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (output): Linear(in_features=64, out_features=1, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
    (activation): LeakyReLU(negative_slope=0.1)
  )
)


In [ ]:
epochs = 500
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = rmse_loss(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    
    with torch.no_grad():
        val_outputs = model(X_test_tensor)
        val_loss = rmse_loss(val_outputs, y_test_tensor)

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')

Epoch [10/500], Train Loss: 87.2844, Val Loss: 86.8879
Epoch [20/500], Train Loss: 83.3919, Val Loss: 82.8668
Epoch [30/500], Train Loss: 78.0402, Val Loss: 77.3494
Epoch [40/500], Train Loss: 70.9567, Val Loss: 70.0915
Epoch [50/500], Train Loss: 62.2643, Val Loss: 61.2653


In [25]:
test_dataset = torch.tensor(test_data.iloc[:, 1:].values, dtype = torch.float32).to('cuda')
submission_pred = model(test_dataset)

In [26]:
submission_pred = submission_pred.cpu().detach().numpy()

In [27]:
submission_pred

array([[81.37665],
       [81.37665],
       [81.37665],
       ...,
       [81.37665],
       [81.37665],
       [81.37665]], dtype=float32)

## Saving submission

In [34]:
def saveSubmission(model, score, name):
    submission_pd = pd.DataFrame()
    pred = model.predict(test_X.iloc[:, :])
    submission_pd = pd.DataFrame({'id' : test_data['id'], 'Price' : pred})
    # filename = f"../data/submissions/{str(name)}_submission_score_{score}_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.csv"
    filename = f"./{str(name)}_submission_score_{score}_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.csv"
    submission_pd.to_csv(filename, index = False)

In [35]:
saveSubmission(hgbr, (mean_squared_error(Y_test, hgbr.predict(X_test)) ** 0.5), 'hgbr')

In [28]:
submission_pd = pd.DataFrame({'id' : test_data['id'], 'Price' : submission_pred.flatten()})

In [29]:
submission_pd.to_csv(f'/kaggle/working/{str("neural_model_5_128_tanh.csv")}', index = False)